<h1>B09705039_劉惟恩_HW04

In [1]:
# 載入所需函示庫
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as smm
import statsmodels
from matplotlib import pyplot as plt
%matplotlib inline
# 設定圖形大小; DPI越大圖越大
plt.rcParams["figure.dpi"] = 120

<h2>Problem 14.90

<h3> The randomized block ANOVA

Hypothesis: <br>
H0: The means of the number of pounds lost between different subjects are the same.  <br>
H1: At least two means differ.

In [2]:
# 讀取資料集
df1 = pd.read_excel('Xr14-90.xlsx')

# 簡單看一下資料集的樣子
# 如果這是你第一次使用這個資料集，使用Excel開啟資料集，
# 並確認程式讀到的跟你在Excel中看到的一樣

print('Head of dataset:')
display(df1.head())

print("Tail of dataset:")
display(df1.tail())

Head of dataset:


,Unnamed: 0,Diet 1,Diet 2,Diet 3,Diet 4
0,Block 1,5,2,6,8
1,Block 2,4,7,8,10
2,Block 3,6,12,9,2
3,Block 4,7,11,16,7
4,Block 5,9,8,15,14


Tail of dataset:


,Unnamed: 0,Diet 1,Diet 2,Diet 3,Diet 4
0,Block 1,5,2,6,8
1,Block 2,4,7,8,10
2,Block 3,6,12,9,2
3,Block 4,7,11,16,7
4,Block 5,9,8,15,14


In [3]:
# wide_to_long假設同樣資料在不同因素下的名稱有相同的字頭，因此我們要改欄位名稱。
# print(df1.columns)
# Note there is an extra space for School D column name.
df1 = df1.rename(columns = {'Unnamed: 0': 'id', 
                            "Diet 1": "pounds_1", 
                            "Diet 2": "pounds_2", 
                            "Diet 3": "pounds_3",
                            "Diet 4": "pounds_4"})  

# 呼叫wide_to_long(); 文件請見: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html
df2 = pd.wide_to_long(df1, ["pounds"], sep = "_", suffix = '\w+', i="id", j="factor").reset_index()
print("Long data")
display(df2)

Long data


,id,factor,pounds
0,Block 1,1,5
1,Block 2,1,4
2,Block 3,1,6
3,Block 4,1,7
4,Block 5,1,9
5,Block 1,2,2
6,Block 2,2,7
7,Block 3,2,12
8,Block 4,2,11
9,Block 5,2,8


In [4]:
results = smf.ols('pounds ~ C(factor) + C(id)', data=df2).fit()
# results.summary()
aov_table = sms.anova_lm(results, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(factor),53.8,3.0,1.652842,0.229629
C(id),102.2,4.0,2.354839,0.112507
Residual,130.2,12.0,NaN,NaN


The p-value for different factor is 0.229629 > 0.01, so we cannot reject the null hypothesis. There is no enough evidence to infer that there are differences between the four diets at the 1% significance level.

<h3>Checking required conditions:

<h4>Condition1: Shapiro-Wilk test of Normality</h4>

H0: The distribution is normal.<br>
H1: The distribution is not normal.

In [5]:
stat, p = stats.shapiro(results.resid)
print('Statistics=%.4f, p=%.4f' % (stat, p))

Statistics=0.9826, p=0.9630


The p-value of Shapiro-Wilk test is 0.9630. As a result, we cannot reject H0. We conclude that the normality assumption is not violated.

<h4>Condition2: Bartlett's Tests for Equal Variance</h4>

H0: All variances are the equal. <br> 
H1: At least two variances are different

In [6]:
df2['resid'] = results.resid
stats.bartlett(df2['resid'][df2['factor'] == 1], 
               df2['resid'][df2['factor'] == 2], 
               df2['resid'][df2['factor'] == 3],
               df2['resid'][df2['factor'] == 4])

BartlettResult(statistic=4.290136711862412, pvalue=0.23179100656950255)

The p-value of Bartlett’s test is 0.23179100656950255. As a result, we cannot reject H0. We conclude that that equal variance assumption is not violated.

<h2>Problem 14.96

<h3> The randomized block ANOVA

Hypothesis: <br>
H0: The means of the numbers of hours of patient care per week are the same.  <br>
H1: At least two means differ.

In [7]:
# 讀取資料集
df1 = pd.read_excel('Xr14-96.xlsx')

# 簡單看一下資料集的樣子
# 如果這是你第一次使用這個資料集，使用Excel開啟資料集，
# 並確認程式讀到的跟你在Excel中看到的一樣

print('Head of dataset:')
display(df1.head())

print("Tail of dataset:")
display(df1.tail())

Head of dataset:


,Age,General,Internal,Surgery,Pediatrics,Obs/Gyn
0,30,54.0,56.7,55.7,58.3,60.6
1,31,54.9,61.8,61.5,50.7,65.2
2,32,55.5,46.6,59.5,56.4,63.2
3,33,37.0,44.6,40.8,51.4,37.3
4,34,52.0,63.8,44.9,52.5,56.6


Tail of dataset:


,Age,General,Internal,Surgery,Pediatrics,Obs/Gyn
31,61,61.3,56.9,70.2,58.0,68.0
32,62,58.7,62.0,65.2,59.3,74.2
33,63,47.7,37.6,44.9,47.3,57.9
34,64,53.5,47.1,61.7,53.5,55.0
35,65,56.2,57.0,58.1,54.1,66.0


In [8]:
# wide_to_long假設同樣資料在不同因素下的名稱有相同的字頭，因此我們要改欄位名稱。
# print(df1.columns)
# Note there is an extra space for School D column name.
df1 = df1.rename(columns = {'Age': 'id', 
                            "General": "hours_g", 
                            "Internal ": "hours_i", 
                            "Surgery": "hours_s",
                            "Pediatrics": "hours_p",
                            "Obs/Gyn": "hours_o"})  

# 呼叫wide_to_long(); 文件請見: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html
df2 = pd.wide_to_long(df1, ["hours"], sep = "_", suffix = '\w+', i="id", j="factor").reset_index()
print("Long data")
display(df2)

Long data


,id,factor,hours
0,30,g,54.0
1,31,g,54.9
2,32,g,55.5
3,33,g,37.0
4,34,g,52.0
...,...,...,...
175,61,o,68.0
176,62,o,74.2
177,63,o,57.9
178,64,o,55.0


In [9]:
results = smf.ols('hours ~ C(factor) + C(id)', data=df2).fit()
# results.summary()
aov_table = sms.anova_lm(results, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(factor),1406.390222,4.0,10.715019,1.322412e-07
C(id),7309.684000,35.0,6.364696,9.352614e-16
Residual,4593.893778,140.0,NaN,NaN


(a) The p-value for different factor is 1.322412e-07 < 0.05, so we reject H0 in favor of H1. We can infer that there are differences in the amount of patient care between medical specialties.

(b) The p-value for different id (blockings) is 9.352614e-16 < 0.05, so we reject H0 in favor of H1. We infer that blocking by age was not appropriate.

<h3>Checking required conditions:

<h4>Condition1: Shapiro-Wilk test of Normality</h4>

H0: The distribution is normal.<br>
H1: The distribution is not normal.

In [10]:
stat, p = stats.shapiro(results.resid)
print('Statistics=%.4f, p=%.4f' % (stat, p))

Statistics=0.9917, p=0.3865


The p-value of Shapiro-Wilk test is 0.3865. As a result, we cannot reject H0. We conclude that the normality assumption is not violated.

<h4>Condition2: Bartlett's Tests for Equal Variance</h4>

H0: All variances are the equal. <br> 
H1: At least two variances are different.

In [11]:
df2['resid'] = results.resid
stats.bartlett(df2['resid'][df2['factor'] == "g"], 
               df2['resid'][df2['factor'] == "i"], 
               df2['resid'][df2['factor'] == "s"],
               df2['resid'][df2['factor'] == "p"],
               df2['resid'][df2['factor'] == "o"])

BartlettResult(statistic=27.229766877854324, pvalue=1.7861856941415005e-05)

The p-value of Bartlett’s test is 1.7861856941415005e-05. As a result, we reject H0 in favor of H1. We conclude that that equal variance assumption is violated in factor.<br>
However, we still can use the ANOVA test to have a rough concept about it.

In [12]:
all_level = df2['id'].unique()
#print("List of all levels: ", all_level)

data_list = []
for alevel in all_level:
    data_list.append(df2['resid'][df2['id'] == alevel])
#print("There are %d samples collected" % len(data_list))

# Conduct Bartlett's Test
stats.bartlett(*data_list)

BartlettResult(statistic=33.91262720038764, pvalue=0.5204732128156567)

The p-value of Bartlett’s test is 0.5204732128156567. As a result, we cannot reject H0. We conclude that that equal variance assumption is not violated in blockings.

<h2>Problem 14.106

In [13]:
# 讀取資料集
df1 = pd.read_excel('Xr14-106.xlsx')

# 簡單看一下資料集的樣子
# 如果這是你第一次使用這個資料集，使用Excel開啟資料集，
# 並確認程式讀到的跟你在Excel中看到的一樣

print('Head of dataset:')
display(df1.head())

print("Tail of dataset:")
display(df1.tail())

Head of dataset:


/Users/liuweien/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Group,Form 1,Form 2,Form 3,Form 4
0,1,43,84,72,81
1,1,57,64,113,152
2,1,53,115,111,95
3,1,58,37,104,71
4,1,72,94,118,138


Tail of dataset:


,Group,Form 1,Form 2,Form 3,Form 4
25,3,93,64,139,158
26,3,130,110,88,92
27,3,109,96,97,38
28,3,122,100,119,158
29,3,124,101,57,74


In [14]:
# wide_to_long假設同樣資料在不同因素下的名稱有相同的字頭，因此我們要改欄位名稱。
# print(df1.columns)
# Note there is an extra space for School D column name.
df1 = df1.rename(columns = {"Group": "group",
                            "Form 1": "time_1", 
                            "Form 2": "time_2", 
                            "Form 3": "time_3",
                            "Form 4": "time_4"})
df1["id"] = df1.index

# 呼叫wide_to_long(); 文件請見: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html
df2 = pd.wide_to_long(df1, ["time"], sep = "_", suffix = '\w+', i="id", j="form").reset_index()

df2 = df2.drop(["id"], axis = 1)

df2['form'] = df2['form'].apply(str)
df2['group'] = df2['group'].apply(str)

print("Long data")
display(df2)

Long data


,form,group,time
0,1,1,43
1,1,1,57
2,1,1,53
3,1,1,58
4,1,1,72
...,...,...,...
115,4,3,158
116,4,3,92
117,4,3,38
118,4,3,158


In [15]:
df2['group_form'] = df2['group'] + "_" + df2['form']

df2.groupby('group_form').describe()

time                                                     
           count   mean        std   min    25%    50%     75%    max
group_form                                                           
1_1         10.0   73.3  28.713334  43.0  54.75   65.0   77.25  138.0
1_2         10.0   79.6  25.461081  37.0  64.00   83.0   93.75  115.0
1_3         10.0  100.6  22.101785  67.0  88.50  101.0  112.50  141.0
1_4         10.0  108.6  27.443680  71.0  89.00  100.0  133.50  152.0
2_1         10.0   95.7  29.382724  56.0  78.25   92.0  116.00  150.0
2_2         10.0   90.5  20.748494  65.0  70.50   91.0  105.75  122.0
2_3         10.0  107.3  24.161724  64.0  91.75  109.5  118.00  142.0
2_4         10.0  110.0  32.731229  54.0  91.75  109.0  129.00  166.0
3_1         10.0  109.9  25.709704  66.0  95.50  115.5  128.50  145.0
3_2         10.0  110.0  31.538336  64.0  97.00  101.0  118.25  186.0
3_3         10.0  103.8  29.682767  57.0  83.50  102.0  125.75  148.0
3_4         10.0  111.5  40.022910  38.0  87.50  109.5  145.00  158.0

<h3> Two-way ANOVA

Hypothesis: <br>

Group: <br>
H0: The means of groups are equal.  <br>
H1: At least two means differ. <br>

Form: <br>
H0: The means of forms are equal.  <br>
H1: At least two means differ. <br>

Interaction: <br>
H0: Group and Form do not interact to affect the mean responses.  <br>
H1: Group and Form do interact to affect the mean responses.

In [16]:
result = smf.ols('time ~ C(group) + C(form) + C(group):C(form)', data=df2).fit()
# results.summary()
aov = sms.anova_lm(result, typ=2)
aov

,sum_sq,df,F,PR(>F)
C(group),6718.716667,2.0,4.112707,0.018994
C(form),6279.866667,3.0,2.562717,0.058572
C(group):C(form),5101.883333,6.0,1.041000,0.402973
Residual,88217.000000,108.0,NaN,NaN


(a) 3 * 4 = 12 treatments.

(b) 2 factors, different brackets and different forms.

(c) brackets: 3 levels, forms: 4 levels.

(d) The p-value for interaction between the two factors is 0.402973 > 0.05, so we cannot reject H0. We can conclude that there is no interaction between the two factors.

(e) The p-value for different forms is 0.058572 > 0.05, so we cannot reject H0. We cannot conclude that differences exist between the four forms.

(f) The p-value for different groups (brackets) is 0.018994 < 0.05, so we reject H0 in favor of H1. We can conclude that taxpayers in different brackets require different amounts of time to complete their tax forms.

<h3>Checking required conditions:

<h4>Condition1: Shapiro-Wilk test of Normality</h4>

H0: The distribution is normal.<br>
H1: The distribution is not normal.

In [17]:
stat, p = stats.shapiro(result.resid)
print('Statistics=%.4f, p=%.4f' % (stat, p))

Statistics=0.9941, p=0.8968


The p-value of Shapiro-Wilk test is 0.8968. As a result, we cannot reject H0. We conclude that the normality assumption is not violated.

<h4>Condition2: Bartlett's Tests for Equal Variance</h4>

H0: All variances are the equal. <br> 
H1: At least two variances are different.

In [18]:
df2['resid'] = result.resid

all_level = df2['group_form'].unique()
#print("List of all levels: ", all_level)

data_list = []
for alevel in all_level:
    data_list.append(df2['resid'][df2['group_form'] == alevel])
#print("There are %d samples collected" % len(data_list))

# Conduct Bartlett's Test
stats.bartlett(*data_list)

BartlettResult(statistic=6.33671175693431, pvalue=0.8499800287699081)

The p-value of Bartlett’s test is 0.8499800287699081. As a result, we cannot reject H0. We conclude that that equal variance assumption is not violated.

<h2>Problem 14.108

In [19]:
# 讀取資料集
df1 = pd.read_excel('Xr14-108.xlsx')

# 簡單看一下資料集的樣子
# 如果這是你第一次使用這個資料集，使用Excel開啟資料集，
# 並確認程式讀到的跟你在Excel中看到的一樣

print('Head of dataset:')
display(df1.head())

print("Tail of dataset:")
display(df1.tail())

Head of dataset:


/Users/liuweien/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Schedule,Drug 1,Drug 2,Drug 3,Drug 4
0,Week,17,24,14,10
1,Week,6,15,9,-1
2,Week,10,10,12,0
3,Week,12,16,0,3
4,Week,14,14,6,-1


Tail of dataset:


,Schedule,Drug 1,Drug 2,Drug 3,Drug 4
5,Two Days,18,-2,20,-2
6,Two Days,9,0,16,7
7,Two Days,17,17,12,10
8,Two Days,21,2,17,6
9,Two Days,15,6,18,7


In [20]:
# wide_to_long假設同樣資料在不同因素下的名稱有相同的字頭，因此我們要改欄位名稱。
# print(df1.columns)
# Note there is an extra space for School D column name.
df1 = df1.rename(columns = {"Schedule": "schedule",
                            "Drug 1": "degree_1", 
                            "Drug 2": "degree_2", 
                            "Drug 3": "degree_3",
                            "Drug 4": "degree_4"})
df1["id"] = df1.index

# 呼叫wide_to_long(); 文件請見: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html
df2 = pd.wide_to_long(df1, ["degree"], sep = "_", suffix = '\w+', i="id", j="drug").reset_index()

df2 = df2.drop(["id"], axis = 1)

df2['schedule'] = df2['schedule'].apply(str)
df2['drug'] = df2['drug'].apply(str)

print("Long data")
display(df2)

Long data


,drug,schedule,degree
0,1,Week,17
1,1,Week,6
2,1,Week,10
3,1,Week,12
4,1,Week,14
5,1,Two Days,18
6,1,Two Days,9
7,1,Two Days,17
8,1,Two Days,21
9,1,Two Days,15


In [21]:
df2['drug_schedule'] = df2['drug'] + "_" + df2['schedule']

df2.groupby('drug_schedule').describe()

degree                                              
               count  mean       std   min   25%   50%   75%   max
drug_schedule                                                     
1_Two Days       5.0  16.0  4.472136   9.0  15.0  17.0  18.0  21.0
1_Week           5.0  11.8  4.147288   6.0  10.0  12.0  14.0  17.0
2_Two Days       5.0   4.6  7.536577  -2.0   0.0   2.0   6.0  17.0
2_Week           5.0  15.8  5.118594  10.0  14.0  15.0  16.0  24.0
3_Two Days       5.0  16.6  2.966479  12.0  16.0  17.0  18.0  20.0
3_Week           5.0   8.2  5.495453   0.0   6.0   9.0  12.0  14.0
4_Two Days       5.0   5.6  4.505552  -2.0   6.0   7.0   7.0  10.0
4_Week           5.0   2.2  4.658326  -1.0  -1.0   0.0   3.0  10.0

<h3> Two-way ANOVA

Hypothesis: <br>

Drugs: <br>
H0: The means of drugs are equal.  <br>
H1: At least two means differ. <br>

Schedules: <br>
H0: The means of schedules are equal.  <br>
H1: At least two means differ. <br>

Interaction: <br>
H0: Drugs and schedules do not interact to affect the mean responses.  <br>
H1: Drugs and schedules do interact to affect the mean responses.

In [22]:
result = smf.ols('degree ~ C(drug) + C(schedule) + C(drug):C(schedule)', data=df2).fit()
# results.summary()
aov = sms.anova_lm(result, typ=2)
aov

,sum_sq,df,F,PR(>F)
C(drug),581.8,3.0,7.711067,0.000516
C(schedule),14.4,1.0,0.572565,0.454777
C(drug):C(schedule),548.6,3.0,7.271040,0.000749
Residual,804.8,32.0,NaN,NaN


(a) different schedules and different drugs

(b) headache index (An index ranging from 0 to 100 was constructed for each patient.) (presented as degree in the dataframe)

(c) schedules: 2 levels, drugs: 4 levels.

(d)<br>
The p-value for different schedules is 0.454777 > 0.05, so we cannot reject H0. We cannot conclude that there are differences in the improvement in the headache index between the two schedules.<br>
The p-value for different drugs is 0.000516 < 0.05, so we reject H0 in favor of H1. We can conclude that there are differences in the improvement in the headache index between the four drug mixtures.<br>
The p-value for interaction between the two factors is 0.000749 < 0.05, so we reject H0 in favor of H1. We can conclude that there is interaction between schedules and drug mixtures.

<h3>Checking required conditions:

<h4>Condition1: Shapiro-Wilk test of Normality</h4>

H0: The distribution is normal.<br>
H1: The distribution is not normal.

In [23]:
stat, p = stats.shapiro(result.resid)
print('Statistics=%.4f, p=%.4f' % (stat, p))

Statistics=0.9786, p=0.6375


The p-value of Shapiro-Wilk test is 0.6375. As a result, we cannot reject H0. We conclude that the normality assumption is not violated.

<h4>Condition2: Bartlett's Tests for Equal Variance</h4>

H0: All variances are the equal. <br> 
H1: At least two variances are different.

In [24]:
df2['resid'] = result.resid

all_level = df2['drug_schedule'].unique()
#print("List of all levels: ", all_level)

data_list = []
for alevel in all_level:
    data_list.append(df2['resid'][df2['drug_schedule'] == alevel])
#print("There are %d samples collected" % len(data_list))

# Conduct Bartlett's Test
stats.bartlett(*data_list)

BartlettResult(statistic=3.592773434657972, pvalue=0.8253035753523124)

The p-value of Bartlett’s test is 0.8253035753523124. As a result, we cannot reject H0. We conclude that that equal variance assumption is not violated.